# House Prices

A large dataset from the UK Office for National Statistics that makes heavy use of irregularly offset headers and whitespace as a visual only means of data demarcation.

## Tutorial Structure

With these example tutorials I'm going to comment heavily and cover nuances in a follow up section (with liberal targetted previews as needed) as it's the easiest way to grapple with new ideas. It may also be worth opening up these notebooks yourself (they're in `./jupyterbook` in the [tidychef](https://github.com/mikeAdamss/tidychef) github repo) so you can run, alter and generally have a play about with this yourself as part of your learning.

We'll cover:

- source data
- requirements, what we're aiming to do here
- show the full script (all logic commented)
- output the selection preview
- nuances (where applicable)
- view the output

This sequencing is necessary as the output for some of the example is **really** long so that necessitates it coming last. If you're viewing this via a jupyter book (i.e on the site) you can navigate between the above sections via your right hand menu.

_Note - these tutorial scripts might seem verbose due all the comments but that's ok (this is a tutorial after all). If you take them out you end up with a fairly succinct and human readable encapsulation of what would otherwise (with existing tools) be a rather convoluted and fragile set of instructions to express._

In virtually all cases I'll make heavy use of `preview` and `bounded` to only look at relevant parts of what can be quite large datasets. Downloads links are provided for the source data.

## Source

In [ ]:
from tidychef import acquire, preview
from tidychef.selection import XlsSelectable

table: XlsSelectable = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/house-prices.xls", tables="Table 11")
preview(table, bounded="A1:M20")

From an xlx source which can be [downloaded here](https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/house-prices.xls).

## Requirements

- We'll take "Year" and "Quarter" from the appropriate values in columns B and C.
- We'll take populated cells on row 4 as "Housing" and we'll strip the "4" notation away.
- We'll take "Area" and "Area Code" from column A (see United Kingdom and K02000001 as the examples).
- We'll call the observations column "Value" and we'll strip any trailing ".0"s.
 
 Note: this is one of the morecomplexbasic examples but its complexity in terms of large amound of simple things. Take you time and work through it logically.

In [ ]:
from tidychef import acquire, preview
from tidychef.direction import up, down, right, left
from tidychef.output import TidyData, Column
from tidychef.selection import XlsSelectable

# Get the data via the acquire api, in this instance we're only interested in table 11 of the spreadsheet
table: XlsSelectable = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/house-prices.xls", tables="Table 11")

# Sweep up some unwanted stuff at the bottom of the table
unwanted = table.column('A').is_numeric().expand_to_box()

# Get the area level - I'm doing something a little sneaky here to avoid regex'ing a large dataset, see nuances.
area = table.excel_ref("A1").fill(down).is_not_blank().shift(up).is_not_blank().label_as("Area") - unwanted

# An area code is always above an area
area_code = area.shift(down).label_as("Area Code")

# - (a) Find the row that's got 1 or more cells with 'Price' written in it
# - (b) Get rid of the blank cells on that row
measure = table.row_containing_strings(['Price']).is_not_blank().label_as("Measure")

# housing is always the line above measure
housing = measure.shift(up)

# observations are numberic values below our "Measure" selections
observations = measure.fill(down).is_not_blank().is_numeric().label_as("Observation")

# We'll select year as number values in column B
year = table.excel_ref('B').is_numeric().label_as("Year")

# years are non black cells to the left of quarters
quarter = year.shift(right).extrude(down(3)).is_not_blank().label_as("Year")

# Create a bounded preview inline but also write the full preview to path
preview(observations, housing, area_code, area, year, quarter, measure, bounded="A1:M20")
preview(observations, housing, area_code, area, year, quarter, measure, path="house-prices.html")

tidy_data = TidyData(
    observations,
    Column(measure.attach_directly(down)),
    Column(housing.attach_directly(down), apply=lambda x: x.rstrip("4")), # Lets strip the notation
    Column(area_code.attach_closest(down)),
    Column(area.attach_closest(down)),
    Column(year.attach_closest(down), apply=lambda x: x.replace(".0", "")), # Present as whole number
    Column(quarter.attach_directly(right)),
    obs_apply = lambda x: x.replace(".0", "") # Present observation as whole number where applicable.
)

tidy_data.to_csv("house-prices.csv")

## Nuances

### How we're getting area - and why

Not everyone likes to use regex and it can have performance issues if you go wild with it, so we're taking a different approach to this. I'll walk through it one step at a time. Here's the full command:

```python
area = table.excel_ref("A1").fill(down).is_not_blank().shift(up).is_not_blank().label_as("Area")
```

Note - for brevity we'll just look at one block the two headers in questions, but remember there are dozens of identical two cell blocks in this column and the commands operate simultaneously on **all** of them.

In [ ]:
# Get not blank cells from Column A -not- counting cell A1 (its got some title information in it
example = table.excel_ref("A1").fill(down).is_not_blank()
preview(example, bounded="A1:M8")

In [ ]:
# Now we shift that selection up by one
example2 = example.shift(up)
preview(example2, bounded="A1:M8")

In [ ]:
# And now we remove blanks again.
example3 = example2.is_not_blank()
preview(example3, bounded="A1:M8")

And voila, a very targetted selection without relying on regex or any other kind of string matching!

It's hard to convey this next point without writing endless example of this kind but a big part of getting the most out of tidychef is learning to leverage command chaining in non obvious ways like this, effectively the more you do this kind of thing the more you learn to in think terms of "visual wrangling".

# Outputs

The full preview can be [viewed here](./house-prices.html).

The tidy data can be [downloaded here](./house-prices.csv) and a full inline preview of the tidydata generated is shown below for those people who'd prefer to scroll.

In [ ]:
print(tidy_data)